In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time

# Configuration
index_url = "https://downloads.dbpedia.org/wiki-archive/Downloads2014.html"
output_dir = "dbpedia_data_en" 
os.makedirs(output_dir, exist_ok=True)

# Uniquement l'anglais
TARGET_SUFFIX = '_en.ttl.bz2'

def download_file(url, dest_folder):
    local_filename = os.path.join(dest_folder, url.split('/')[-1])
    
    if os.path.exists(local_filename):
        print(f"[-] Déjà présent : {local_filename}")
        return

    print(f"[+] Téléchargement de {url}...")
    try:
        with requests.get(url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(local_filename, 'wb') as f:
                for chunk in r.iter_content(chunk_size=1024*1024):
                    if chunk:
                        f.write(chunk)
        print(f"    OK")
        # On garde une pause de 2s pour respecter le serveur de Mannheim
        time.sleep(2) 
    except Exception as e:
        print(f"    ERREUR sur {url} : {e}")
        time.sleep(5)

def main():
    print("Analyse de la page d'index pour les fichiers anglais...")
    try:
        response = requests.get(index_url)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # On ne prend que ce qui finit par _en.ttl.bz2
        links_to_download = [a['href'] for a in soup.find_all('a', href=True) 
                             if a['href'].endswith(TARGET_SUFFIX)]
        
        print(f"Nombre de fichiers anglais trouvés : {len(links_to_download)}")
        
        for i, link in enumerate(links_to_download):
            print(f"\nFichier {i+1}/{len(links_to_download)}")
            download_file(link, output_dir)
            
    except Exception as e:
        print(f"Erreur lors de la lecture de l'index : {e}")

if __name__ == "__main__":
    main()

Analyse de la page d'index pour les fichiers anglais...
Nombre de fichiers anglais trouvés : 30

Fichier 1/30
[+] Téléchargement de http://data.dws.informatik.uni-mannheim.de/dbpedia/2014/en/instance_types_en.ttl.bz2...
    OK

Fichier 2/30
[+] Téléchargement de http://data.dws.informatik.uni-mannheim.de/dbpedia/2014/en/mappingbased_properties_en.ttl.bz2...
    OK

Fichier 3/30
[+] Téléchargement de http://data.dws.informatik.uni-mannheim.de/dbpedia/2014/en/specific_mappingbased_properties_en.ttl.bz2...
    OK

Fichier 4/30
[+] Téléchargement de http://data.dws.informatik.uni-mannheim.de/dbpedia/2014/en/labels_en.ttl.bz2...
    OK

Fichier 5/30
[+] Téléchargement de http://data.dws.informatik.uni-mannheim.de/dbpedia/2014/en/short_abstracts_en.ttl.bz2...
    ERREUR sur http://data.dws.informatik.uni-mannheim.de/dbpedia/2014/en/short_abstracts_en.ttl.bz2 : HTTPSConnectionPool(host='data.dws.informatik.uni-mannheim.de', port=443): Read timed out.

Fichier 6/30
[+] Téléchargement de http:/

In [ ]:
import re
import urllib.parse

input_path = "/Users/adammounir/graphdb-import/labels_en.ttl"
output_path = "/Users/adammounir/graphdb-import/labels_en_final.nt" 
print("Lancement du nettoyage agressif (1.26 Go)...")

# Liste des caractères à ne PAS encoder dans une IRI
safe_chars = "/:?#=@&"

def fix_iri(match):
    iri = match.group(1)
    # On encode tout ce qui est bizarre tout en gardant la structure de l'URL
    fixed = urllib.parse.quote(iri, safe=safe_chars)
    return f"<{fixed}>"

iri_pattern = re.compile(r'<(.*?)>')

with open(input_path, 'r', encoding='utf-8', errors='replace') as infile, \
     open(output_path, 'w', encoding='utf-8') as outfile:
    
    for count, line in enumerate(infile):
        if line.startswith('#') or not line.strip():
            continue
        
        # Nettoyage des adresses (IRIs)
        new_line = iri_pattern.sub(fix_iri, line)
        outfile.write(new_line)
        
        if count % 500000 == 0 and count > 0:
            print(f"Progression : {count} lignes traitées...")

print(f"TERMINE ! Fichier prêt : {output_path}")

Lancement du nettoyage agressif (1.26 Go)...
Progression : 500000 lignes traitées...
Progression : 1000000 lignes traitées...
Progression : 1500000 lignes traitées...
Progression : 2000000 lignes traitées...
Progression : 2500000 lignes traitées...
Progression : 3000000 lignes traitées...
Progression : 3500000 lignes traitées...
Progression : 4000000 lignes traitées...
Progression : 4500000 lignes traitées...
Progression : 5000000 lignes traitées...
Progression : 5500000 lignes traitées...
Progression : 6000000 lignes traitées...
Progression : 6500000 lignes traitées...
Progression : 7000000 lignes traitées...
Progression : 7500000 lignes traitées...
Progression : 8000000 lignes traitées...
Progression : 8500000 lignes traitées...
Progression : 9000000 lignes traitées...
Progression : 9500000 lignes traitées...
Progression : 10000000 lignes traitées...
Progression : 10500000 lignes traitées...
TERMINE ! Fichier prêt : /Users/adammounir/graphdb-import/labels_en_final.nt


In [8]:
import re
import urllib.parse

input_path = "/Users/adammounir/graphdb-import/redirects_transitive_en.ttl"
output_path = "/Users/adammounir/graphdb-import/redirects_transitive_en_final.nt" 
print("Lancement du nettoyage agressif (1.26 Go)...")

# Liste des caractères à ne PAS encoder dans une IRI
safe_chars = "/:?#=@&"

def fix_iri(match):
    iri = match.group(1)
    # On encode tout ce qui est bizarre tout en gardant la structure de l'URL
    fixed = urllib.parse.quote(iri, safe=safe_chars)
    return f"<{fixed}>"

iri_pattern = re.compile(r'<(.*?)>')

with open(input_path, 'r', encoding='utf-8', errors='replace') as infile, \
     open(output_path, 'w', encoding='utf-8') as outfile:
    
    for count, line in enumerate(infile):
        if line.startswith('#') or not line.strip():
            continue
        
        # Nettoyage des adresses (IRIs)
        new_line = iri_pattern.sub(fix_iri, line)
        outfile.write(new_line)
        
        if count % 500000 == 0 and count > 0:
            print(f"Progression : {count} lignes traitées...")

print(f"TERMINE ! Fichier prêt : {output_path}")

Lancement du nettoyage agressif (1.26 Go)...
Progression : 500000 lignes traitées...
Progression : 1000000 lignes traitées...
Progression : 1500000 lignes traitées...
Progression : 2000000 lignes traitées...
Progression : 2500000 lignes traitées...
Progression : 3000000 lignes traitées...
Progression : 3500000 lignes traitées...
Progression : 4000000 lignes traitées...
Progression : 4500000 lignes traitées...
Progression : 5000000 lignes traitées...
Progression : 5500000 lignes traitées...
Progression : 6000000 lignes traitées...
TERMINE ! Fichier prêt : /Users/adammounir/graphdb-import/redirects_transitive_en_final.nt
